In [ ]:
import ccxt
import json
import pprint
import numpy as np
import pandas as pd

In [ ]:
# choices: okx, binanceusdm, bitget, bybit, kucoinfutures
exchange = "kucoinfutures"
cc = getattr(ccxt, exchange)()

In [ ]:
info = cc.fetch_markets()

In [ ]:
tickers = cc.fetch_tickers()

In [ ]:
min_costs = {}
for x in info:
    symbol = x["symbol"]
    if symbol.endswith("USDT"):
        if x["type"] != "spot":
            if exchange in ["okx", "bitget", "kucoinfutures"]:
                ticker_symbol = symbol.replace(":USDT", "")
            else:
                ticker_symbol = symbol
            if ticker_symbol in tickers:
                if exchange == "bitget":
                    min_cost = 5.0
                    c_mult = 1.0
                    min_qty = float(x["info"]["minTradeNum"])
                else:
                    min_cost = (
                        0.0 if x["limits"]["cost"]["min"] is None else x["limits"]["cost"]["min"]
                    )
                    c_mult = 1.0 if x["contractSize"] is None else x["contractSize"]
                    min_qty = (
                        0.0 if x["limits"]["amount"]["min"] is None else x["limits"]["amount"]["min"]
                    )
                if exchange == "kucoinfutures":
                    last_price = float(tickers[ticker_symbol]["info"]["last"])
                else:
                    last_price = tickers[ticker_symbol]["last"]
                min_costs[ticker_symbol] = max(min_cost, min_qty * c_mult * last_price)

In [ ]:
sorted(min_costs.items(), key=lambda x: x[1], reverse=True)